## **💸 Best BTP Finder**

Questo notebook permette di effettuare lo scraping dal sito web [www.rendimentibtp.it](https://www.rendimentibtp.it/) e di filtrare secondo i propri criteri.

Eseguire tutto il notebook e poi eseguire con l'ultima cella per interagire con i dati reperiti.

Modificare i parametri:
   - `MIN_CEDOLA_NETTA` = 2.25 (valore percentuale 2.0 -> 2.0%)
   - `MIN_DURATA_ANNI` = 2 (in anni)
   - `MAX_DURATA_ANNI` = 13 (in anni)
   - `MIN_RENDIMENTO_NETTO` = 3.40 (valore percentuale 2.0 -> 2.0%)
   - `MAX_PREZZO` = 97 

In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

def parse_html_table_to_dataframe():

    # Make a request to the website and get the HTML content
    url = 'https://www.rendimentibtp.it/' # replace with the actual URL of the website you are scraping
    response = requests.get(url)
    html_content = response.content

    # Parse the HTML content with BeautifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')

    # Find all the table rows (tr) in the HTML content
    table_rows = soup.find_all('tr')

    # Create empty lists to store the data for each column
    isins = []
    titoli = []
    durate = []
    scadenze = []
    tassi = []
    prezzi = []
    cedole_netto = []

    # Loop through each table row (skipping the first one, which contains the table headers)
    for row in table_rows[1:]:
        # Find all the table cells (td) in the row
        cells = row.find_all('td')
        
        # Extract the data from each cell and append it to the corresponding list
        isin = cells[0].a.text
        isins.append(isin)
        
        titolo = cells[1].text
        titoli.append(titolo)
        
        durata = cells[2].text
        durate.append(durata)
        
        scadenza = cells[3].text
        scadenze.append(scadenza)
        
        tasso = cells[4].text
        tassi.append(tasso)
        
        prezzo = cells[5].text
        prezzi.append(prezzo)
        
        cedola_netto = cells[6].text
        cedole_netto.append(cedola_netto)
        

    # Create a pandas DataFrame with the data
    data = {
        'isin': isins,
        'titolo': titoli,
        'durata': durate,
        'scadenza': scadenze,
        'tasso': tassi,
        'prezzo': prezzi,
        'cedola netta': cedole_netto
    }
    df = pd.DataFrame(data)
    return df

In [44]:
df = parse_html_table_to_dataframe()

df['prezzo'] = df['prezzo'].str.replace(',', '.').astype(float)
df['durata'] = df['durata'].str.replace(',', '.').astype(float)
df['cedola netta'] = df['cedola netta'].str.replace(',', '.').str.replace('%', '').astype(float)
df['tasso'] = df['tasso'].str.replace(',', '.').str.replace('%', '').astype(float)
df['cedola netta'] = 100/df['prezzo'] * df['tasso'] * 0.875
df['rendimento netto'] = round((((100/df['prezzo'])-1)*100/df['durata'])*0.875 + df['cedola netta'],2)
df['cedola netta'] = round(df['cedola netta'],2)

df

,isin,titolo,durata,scadenza,tasso,prezzo,cedola netta,rendimento netto
0,IT0005438004,1.5-BTP-01AP45,21.75,01.04.2045,1.50,61.58,2.13,4.64
1,IT0005425233,1.7-BTP-01ST51,28.17,01.09.2051,1.70,59.73,2.49,4.58
2,IT0005480980,2.15-BTP-01ST52,29.17,01.09.2052,2.15,65.21,2.88,4.49
3,IT0005441883,2.15-BTP-01MZ72,48.67,01.03.2072,2.15,59.22,3.18,4.41
4,IT0005433195,0.95-BTP-01MZ37,13.67,01.03.2037,0.95,67.05,1.24,4.39
...,...,...,...,...,...,...,...,...
93,IT0004356843*,4.75-BTP-1AG23,0.09,01.08.2023,4.75,100.12,4.15,2.99
94,IT0005413684,0.30-BTP-15AG23,0.12,15.08.2023,0.30,99.63,0.26,2.97
95,IT0005004426,2.35-BTP-01ST24,1.17,01.09.2024,2.35,100.97,2.04,1.32
96,IT0004735152*,3.1-BTP-15ST26,3.21,15.09.2026,3.10,105.04,2.58,1.27


In [55]:
def found_best_btps(df, min_cedola, min_rendimento, min_durata, max_durata, max_prezzo):
    # filter by cedola netta > min_cedola, durata > min_durata, and durata < max_durata
    filtered_df = df[(df['cedola netta'] > min_cedola) & 
                     (df['durata'] > min_durata) & 
                     (df['durata'] < max_durata) & 
                     (df['prezzo'] < max_prezzo) &
                     (df['rendimento netto'] > min_rendimento)]
    # sort by rendimento netto
    sorted_df = filtered_df.sort_values(by='rendimento netto', ascending=False)

    print(f"💸 Trovati {len(sorted_df)} BTP che soddisfano i tuoi criteri:")
    print(f"    > Rendimento netto >= {min_rendimento}%")
    print(f"    > Cedola netta {min_durata}-{max_durata} anni")
    print(f"    > Maturity >= {min_cedola}%")
    print(f"    > Prezzo <= {max_prezzo}%")
    return sorted_df

In [56]:
MIN_CEDOLA_NETTA = 2.25
MIN_DURATA_ANNI = 2
MAX_DURATA_ANNI = 13
MIN_RENDIMENTO_NETTO = 3.40
MAX_PREZZO = 97

found_best_btps(df,
                min_cedola=MIN_CEDOLA_NETTA,
                max_durata=MAX_DURATA_ANNI,
                min_durata=MIN_DURATA_ANNI,
                min_rendimento=MIN_RENDIMENTO_NETTO,
                max_prezzo=MAX_PREZZO)

💸 Trovati 3 BTP che soddisfano i tuoi criteri:
    > Rendimento netto >= 3.4%
    > Cedola netta 2-13 anni
    > Maturity >= 2.25%
    > Prezzo <= 97%


,isin,titolo,durata,scadenza,tasso,prezzo,cedola netta,rendimento netto
22,IT0005358806,3.35-BTP-01MZ35,11.67,01.03.2035,3.35,92.66,3.16,3.76
24,IT0005240350,2.45-BTP-01ST33,10.17,01.09.2033,2.45,86.99,2.46,3.75
30,IT0005494239,2.5-BTP-01DC32,9.42,01.12.2032,2.50,88.72,2.47,3.65
